# Classification Example with Thetis

Thetis can evaluate the AI safety of classifier models.
In a first step, we demonstrate how to evaluate and rate your AI model using a basic classification
example from [scikit-learn](https://scikit-learn.org/).
The instructions below should be easy do adapt to your own use-case.

## Set Up the Environment

In a first step, you need to install Thetis by using pip:

```shell
$ pip install thetis
```

Next, you need to obtain a license in order to use Thetis.

For the current example, you can use the *demo license* located within the same directory as this notebook.
This license only works for our demonstration data set with the exact configuration provided in this notebook.
Use the license file [demo_license_classification.dat](https://raw.githubusercontent.com/EFS-OpenSource/Thetis/main/examples/demo_license_classification.dat).

A customized *full license*, enabling you to run Thetis with your own data sets and settings, is available our [Subscription Page](https://efs-opensource.github.io/Thetis/subscription.html).

Place the license file either in the working directory of your application or at:

- Windows: `<User>/AppData/Local/Thetis/license.dat`
- Unix: `~/.local/thetis/license.dat`

## Increase Logging Verbosity

For detailed runtime information about Thetis, run the following cell to add a logging handler to the Thetis logger to increase verbosity of the application.

In [ ]:
import logging
import sys

# Configure root logger as catch-all logging config
logger = logging.getLogger("Thetis")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stderr)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

## Prepare Example Data and Model

To start with our basic classification example, we need to load some data. In this tutorial, we use the
[Adult data set](https://www.openml.org/search?type=data&sort=runs&id=179&status=active).
This data set demonstrates a prediction task that determines whether a person earns over 50K a year.
Let us now load the data set using tools from the scikit-learn library.

*Note:* If your machine is behind a proxy server, downloading the example data as shown here may not work.
If that is the case for you, check out our [detection example](detection.ipynb) instead.

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_openml

testset_size = 10000

# use "fetch_openml" by scikit-learn to load "Adult" dataset from OpenML
dataset, target = fetch_openml(data_id=1590, return_X_y=True, parser="auto")

df_train, df_test = dataset.iloc[:-testset_size], dataset.iloc[-testset_size:]
target_train, target_test = target.iloc[:-testset_size], target.iloc[-testset_size:]

# drop columns with sensitive attributes from classifier input and convert categorical attributes to one-hot
df_train_cleared = df_train.drop(columns=["education", "race", "sex", "native-country", "relationship", "marital-status"])
df_test_cleared = df_test.drop(columns=["education", "race", "sex", "native-country", "relationship", "marital-status"])

# convert categorical columns to class codes with integer representation
categorical_columns = ["workclass", "occupation"]
df_train_cleared[categorical_columns] = df_train_cleared[categorical_columns].apply(lambda col: pd.Categorical(col).codes)
df_test_cleared[categorical_columns] = df_test_cleared[categorical_columns].apply(lambda col: pd.Categorical(col).codes)

This yields two [Pandas](https://pandas.pydata.org/) data frames with a reduced set of information.

In the next step, we train a simple Random Forest classifier on the training data using scikit-learn.
Furthermore, we make predictions on the test data using the trained model:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# initialize a Random Forest classifier and fit to training data
classifier = RandomForestClassifier(verbose=True)
classifier.fit(pd.get_dummies(df_train_cleared), target_train)

# finally, make predictions on the validation data set
confidence = classifier.predict_proba(pd.get_dummies(df_test_cleared))
labels = classifier.predict(pd.get_dummies(df_test_cleared))

## Represent Data as Pandas DataFrames

Thetis expects two Pandas data frames to run an evaluation:

* Annotations: `pd.DataFrame` with ground-truth information about the data set. The column `target` is required, holding
  the ground-truth target information. Furthermore, columns for sensitive attributes are expected that have been
  configured for the AI Fairness evaluation.
* Predictions: `pd.DataFrame` with the AI predictions for each sample in the data set. The columns `labels` and
  `confidence` are required, holding information about the predicted label and the respective prediction
  probability (model uncertainty or confidence).

Note that the indices of the data frames for annotations and predictions must match.

In [ ]:
# use sensitive attributes during safety evaluation
annotations = pd.DataFrame({"target": target_test, "race": df_test["race"], "sex": df_test["sex"]})
predictions = pd.DataFrame({"labels": labels, "confidence": confidence[:, 1]}, index=annotations.index)

Optionally, you can read/write the `pd.DataFrame` instances in CSV format:

In [ ]:
# optional: store prediction and ground-truth data on disk
annotations.to_csv("adult_annotations.csv")
predictions.to_csv("adult_predictions.csv")

# optional: load prediction and ground-truth data from disk
# important: specify "index_col" since Thetis matches the predictions/annotations by their indices
loaded_annotations = pd.read_csv("adult_annotations.csv", index_col=0)
loaded_predictions = pd.read_csv("adult_predictions.csv", index_col=0)

## Run AI Safety Evaluation with Thetis

For all the details of Thetis configuration, see section [Configuration](https://efs-opensource.github.io/Thetis/configuration.html). You can download the [demo configuration file](https://raw.githubusercontent.com/EFS-OpenSource/Thetis/main/examples/demo_config_classification.yaml) for the current example from the this repository or from [here](https://thetishostedfiles.blob.core.windows.net/demofiles/thetis_demo_classification.zip).

Thetis returns its findings, the final rating and recommendations for mitigation strategies as a JSON-like dictionary. Below, we capture the dictionary as `result` and can access the different evaluation aspects:

* `result[<task>]['rating_score']` for the rating score of the selected task (e.g., 'fairness' or 'uncertainty').
* `result[<task>]['recommendations']` for the recommendations to mitigate possible issues of the selected task.
* `result[<task>]['rating_enum']` for a categorization of the actual aspect into `'GOOD'`, `'MEDIUM'`,
  or `'BAD'` depending on the rating score.

In [ ]:
from thetiscore import thetis


result = thetis(
   config="demo_config_classification.yaml",
   annotations=annotations,
   predictions=predictions,
   output_dir="./output",
   license_file_path="demo_license_classification.dat"
)